# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import matplotlib.pyplot as plt
import json
from holoviews.operation.datashader import datashade

import warnings
warnings.filterwarnings("ignore")
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port douglas,-16.4833,145.4667,295.72,78,56,4.56,AU,1723075898
1,1,bilibino,68.0546,166.4372,285.64,44,1,3.51,RU,1723075899
2,2,selebi-phikwe,-21.9790,27.8498,286.23,76,0,2.12,BW,1723075901
3,3,iqaluit,63.7506,-68.5145,284.00,58,40,0.00,CA,1723075902
4,4,waitangi,-43.9535,-176.5597,285.27,77,71,3.58,NZ,1723075903


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = "Humidity", 
    color = "City"  
)

# Display the plot
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Max Temp 65F(291) - 85F(303)
# Humidity 68%
# Cloudiness 40%
# wind speed 6 mph

vacation_cities_df = city_data_df[
    (city_data_df['Max Temp'] > 291) & (city_data_df['Max Temp'] < 303) & 
    (city_data_df['Humidity'] < 68) & (city_data_df['Humidity'] > 60) & 
    (city_data_df['Cloudiness'] < 40) & (city_data_df['Cloudiness'] > 20) & 
    (city_data_df['Wind Speed'] < 6) & (city_data_df['Wind Speed'] > 2)]

# Drop rows with null values
vacation_cities_df.dropna(inplace=True)

# Display sample data
vacation_cities_df

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
119,119,torghay,49.6339,63.4981,296.37,63,38,3.09,KZ,1723076057
218,218,berdyansk,46.7503,36.7944,294.19,64,33,3.88,UA,1723076200


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_cities_df[['City', 'Country', 'Latitude', 'Longitude', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = " "

# Display sample data
hotel_df

,City,Country,Latitude,Longitude,Humidity,Hotel Name
119,torghay,KZ,49.6339,63.4981,63,
218,berdyansk,UA,46.7503,36.7944,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"apiKey":geoapify_key}
    # Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Latitude"]
    longitude = hotel_df.loc[index, "Longitude"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
torghay - nearest hotel: No hotel found
berdyansk - nearest hotel: No hotel found


,City,Country,Latitude,Longitude,Humidity,Hotel Name
119,torghay,KZ,49.6339,63.4981,63,No hotel found
218,berdyansk,UA,46.7503,36.7944,64,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:


map_plot_2 = hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 700,
    size = 300, 
    color = "Hotel Name"  
)


# Display the plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (Hotel Name)